In [ ]:
# %load main.py
#!/usr/bin/env python3
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))


def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    graph = tf.get_default_graph()
    input_tensor = graph.get_tensor_by_name(vgg_input_tensor_name)
    keep_prob = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3_out = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4_out = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7_out = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    
    return input_tensor, keep_prob, layer3_out, layer4_out, layer7_out
tests.test_load_vgg(load_vgg, tf)


def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer3_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer7_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    #scale layers 3 and 4
    layer3_scaled = tf.scalar_mul(0.0001, vgg_layer3_out)
    layer3_conv1x1 = tf.layers.conv2d(layer3_scaled, num_classes, 1, (1,1), padding='same',                                kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3), name='layer3_1x1conv')
    
    layer4_scaled = tf.scalar_mul(0.01, vgg_layer4_out)
    layer4_conv1x1 = tf.layers.conv2d(layer4_scaled, num_classes, 1, (1,1), padding='same',                                kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3), name='layer4_1x1conv')

    #1x1 convolutions to obtain desired number of num_classes
    conv1x1_1 = tf.layers.conv2d(vgg_layer7_out, num_classes, 1, (1,1), padding='same',                                      kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3), name='layer7_1x1conv')
   
    deconv_1 = tf.layers.conv2d_transpose(conv1x1_1, num_classes, 4, 2, padding='same', 
                                                kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3), name='layer7_deconv')
    
    skip_1 = tf.add(deconv_1, layer4_conv1x1, name='layer_7_add_4')

    deconv_2 = tf.layers.conv2d_transpose(skip_1, num_classes, 4, 2, padding='same', 
                                                kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3), name='layer_74_deconv')
    
    skip_2 = tf.add(deconv_2, layer3_conv1x1, name='layer_7_add_3')

    output_layer = tf.layers.conv2d_transpose(skip_2, num_classes, 16, 8, padding='same',
                                              kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3), name='output_layer')
        
    # TODO: Implement function
    return output_layer
tests.test_layers(layers)


def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    logits = tf.reshape(nn_last_layer, (-1, num_classes), name='logits')
    
    # TODO: Implement function
    reg_loss = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES) #regularization loss
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=nn_last_layer, labels=correct_label), name='cross_entropy_loss')
    overall_loss = tf.add(1.0*sum(reg_loss), cross_entropy_loss, name='total_loss')
    
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
    training_operation = optimizer.minimize(overall_loss, name='train_op')

    return logits, training_operation, overall_loss

tests.test_optimize(optimize)


def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    # TODO: Implement function
    sess.run(tf.global_variables_initializer())
    
    print("Training...")
    print()

    for i in range(epochs):
        #full_images, full_labels = get_batches_fn(-1)
        print("EPOCH {} ...".format(i+1))
        for image, label in get_batches_fn(batch_size):

            optimizer, loss = sess.run([train_op, cross_entropy_loss], 
                               feed_dict={input_image: image, correct_label: label, keep_prob: 0.5})
            print("Loss: = {:.3f}".format(loss))
        
            print()

tests.test_train_nn(train_nn)


def run():
    num_classes = 2
    image_shape = (160, 576)
    data_dir = '/models/data'
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)

    epochs = 14
    batch_size = 32

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/

    
    correct_label = tf.placeholder(tf.int32, [None, None, None, num_classes], name='correct_label')
    learning_rate = 0.0005

    with tf.Session() as sess:
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

        # TODO: Build NN using load_vgg, layers, and optimize function
        input_image, keep_prob, vgg_layer3_out, vgg_layer4_out, vgg_layer7_out = load_vgg(sess, vgg_path)

        nn_last_layer = layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes)

        logits, train_op, cross_entropy_loss = optimize(nn_last_layer, correct_label, learning_rate, num_classes)
        
        # TODO: Train NN using the train_nn function
        train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate)
        
        saver = tf.train.Saver()
        tf.train.write_graph(sess.graph_def, '/models', 'train.pb', as_text=False)
        saver.save(sess, '/models/model')
        
        # TODO: Save inference data using helper.save_inference_samples
        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)
        
        #save graph
        # OPTIONAL: Apply the trained model to a video


if __name__ == '__main__':
    run()


TensorFlow Version: 1.2.1
Tests Passed


/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: No GPU found. Please use a GPU to train your neural network.
  app.launch_new_instance()


Tests Passed
Tests Passed
INFO:tensorflow:Restoring parameters from b'/models/data/vgg/variables/variables'


In [1]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests
from tensorflow.python.platform import gfile

In [ ]:
num_classes = 2
image_shape = (160, 576)
data_dir = '/models/data'
runs_dir = './runs'
model_name = "/models/frozen_graph.pb"
variables_name = "/models/model.ckpt"
tests.test_for_kitti_dataset(data_dir)

input_tensor_name = 'image_input:0'
keep_prob_tensor_name = 'keep_prob:0'
logits_tensor_name = "logits:0"

with tf.Session() as sess:

    graph_def = tf.GraphDef()
    with gfile.FastGFile(model_name,'rb') as f:
        graph_def.ParseFromString(f.read())
        
    tf.import_graph_def(graph_def, name='')
    '''
    saver = tf.train.Saver()
    saver.restore(sess, variables_name)
    
    new_saver = tf.train.import_meta_graph('/models/model.meta')
    new_saver.restore(sess, '/models/model')
    '''
    input_image = sess.graph.get_tensor_by_name(input_tensor_name)
    keep_prob = sess.graph.get_tensor_by_name(keep_prob_tensor_name)
    logits = sess.graph.get_tensor_by_name(logits_tensor_name)
    
    
    helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)


Tests Passed
Training Finished. Saving test images to: ./runs/1544968283.2521915


In [ ]:
saved_model_cli show --dir /models/data/vgg --all